### 1. 서울시 동 경계 지도 준비하기

In [14]:
import json

# EMD_Seoul.geojson 파일을 geo_seoul로 불러옴.
geo_seoul = json.load(open('EMD_Seoul.geojson', 'r', encoding='utf-8'))

# geo_seoul의 첫 번째 요소에서 'properties'와 'geometry'를 출력함.
# 'properties'는 행정 구역의 속성 정보(코드, 이름 등)를 담음.
# 'geometry'는 위도, 경도 좌표를 이용한 실제 지도 모양 정보를 담음.
print("첫 번째 지역의 속성 정보 (properties)")
print(geo_seoul['features'][0]['properties'])
print("\n 첫 번째 지역의 좌표 정보 (geometry)")
print(geo_seoul['features'][0]['geometry'])

첫 번째 지역의 속성 정보 (properties)
{'BASE_DATE': '20200630', 'ADM_DR_CD': '1101053', 'ADM_DR_NM': '사직동', 'OBJECTID': '1'}

 첫 번째 지역의 좌표 정보 (geometry)
{'type': 'MultiPolygon', 'coordinates': [[[[126.97398562200112, 37.578232670691676], [126.97400165856983, 37.578091598158124], [126.97401347517625, 37.57797124764524], [126.97402588957173, 37.57786305895336], [126.97403969022386, 37.57773527607586], [126.97405282759235, 37.57763245751229], [126.97405724608929, 37.57759228442397], [126.97406454937413, 37.5775223449396], [126.97406606875553, 37.57751912550699], [126.97406875866477, 37.57749510583154], [126.97406829700772, 37.5774925030879], [126.97406834313652, 37.57749109719524], [126.97415332533838, 37.57749252255686], [126.97415369692854, 37.57749636934987], [126.97414192870245, 37.577607692520026], [126.97412902832193, 37.57772060121065], [126.97417294290439, 37.57749796032778], [126.97417276162724, 37.57749252680757], [126.9742642955674, 37.577008320942475], [126.97428742236072, 37.5768863505

### 2. 서울시 동별 외국인 인구 데이터 준비하기

In [10]:
import pandas as pd

# Foreigner_EMD_Seoul.csv 파일을 foreigner로 불러옴.
foreigner = pd.read_csv('Foreigner_EMD_Seoul.csv')

# head()로 데이터의 앞부분 5개를 확인함.
print(" foreigner 데이터 확인 (head) ")
print(foreigner.head())

# info()로 데이터의 기본 정보를 확인함. (컬럼별 데이터 타입, 누락 값 등)
print("\nforeigner 데이터 정보 (info)")
foreigner.info()

# 지도 데이터의 행정동 코드('ADM_DR_CD')와 데이터 타입을 맞추기 위해
# foreigner의 'code' 컬럼을 문자(string) 타입으로 변경함.
foreigner['code'] = foreigner['code'].astype(str)

print("\n 'code' 타입 변경 후 foreigner 데이터 정보 (info)")
foreigner.info()

 foreigner 데이터 확인 (head) 
      code region    pop
0  1101053    사직동  418.0
1  1101054    삼청동  112.0
2  1101055    부암동  458.0
3  1101056    평창동  429.0
4  1101057    무악동  102.0

foreigner 데이터 정보 (info)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 0 to 3489
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    3490 non-null   int64  
 1   region  3490 non-null   object 
 2   pop     3486 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 81.9+ KB

 'code' 타입 변경 후 foreigner 데이터 정보 (info)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 0 to 3489
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    3490 non-null   object 
 1   region  3490 non-null   object 
 2   pop     3486 non-null   float64
dtypes: float64(1), object(2)
memory usage: 81.9+ KB


### 3. 단계 구분도 만들기

In [13]:
import folium

# 인구 수(pop)를 기준으로 8개 계급 구간을 정하기 위해 quantile()을 사용.
# 0%, 20%, 40%, ... 100%에 해당하는 값을 기준으로 구간을 나눔.
bins = list(foreigner['pop'].quantile([0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))

# 지도의 중심 좌표(서울)와 확대 레벨, 타일 종류를 설정하여 기본 배경 지도를 생성함.
map_seoul = folium.Map(location=[37.56, 127], zoom_start=12, tiles='cartodbpositron')

# folium.Choropleth를 이용해 단계 구분도를 생성함.
folium.Choropleth(
    geo_data=geo_seoul,                           # 지도 경계 데이터
    data=foreigner,                               # 시각화할 데이터
    columns=['code', 'pop'],                      # 데이터에서 사용할 컬럼 (연결 기준, 수치)
    key_on='feature.properties.ADM_DR_CD',        # 지도 데이터와 연결할 키 값
    fill_color='Blues',                           # 채우기 색상
    nan_fill_color='White',                       # 데이터가 없는 지역의 색상
    fill_opacity=0.7,                             # 색상 투명도
    line_opacity=0.2,                             # 경계선 투명도
    bins=bins,                                    # 계급 구간
    legend_name='Foreigner Population'            # 범례 이름
).add_to(map_seoul)

# 완성된 지도를 seoul_foreigner_map.html 파일로 저장함.
map_seoul.save('seoul_foreigner_map.html')
